<a href="https://colab.research.google.com/github/King-Rian/Project-3/blob/main/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()


Saving emojis.json.zip to emojis.json.zip


In [4]:
from google.colab import files
import zipfile
import json
import pandas as pd

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Extract the ZIP file
with zipfile.ZipFile('emojis.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Step 3: Load the extracted JSON file
emoji_file = 'emojis.json'

# Using pandas
emoji_df = pd.read_json(emoji_file)
print("Dataset loaded successfully. Sample data:")
print(emoji_df.head())


Saving emojis.json.zip to emojis.json (1).zip
Dataset loaded successfully. Sample data:
                                            category  \
0  Miscellaneous Symbols And Pictographs -> Emoji...   
1  Miscellaneous Symbols And Pictographs -> Emoji...   
2                                               None   
3  Miscellaneous Symbols And Pictographs -> Emoji...   
4                                               None   

                                            keywords  \
0  [dark skin tone, hand, forbidden, gesture, wom...   
1                     [dark skin tone, woman, guard]   
2                 [racing, running, woman, marathon]   
3  [gymnastics, medium-light skin tone, woman, ca...   
4                                      [woman, golf]   

                                          definition  \
0  The Woman Gesturing Not OK, Type-6 emoji is a ...   
1  The Female Guard, Type-6 emoji is a sequence o...   
2  The female version of the ?? Runner emoji. The...   
3  The Woman D

In [5]:
# Step 4: Preprocess the data to map context words to emojis
def map_phrases_to_emojis(df):
    mapping = {}
    for _, row in df.iterrows():
        for word in row.get("context_words", []):  # Safely access context_words
            mapping[word] = row["emoji"]
    return mapping

emoji_mapping = map_phrases_to_emojis(emoji_df)
print("Sample emoji mapping:", list(emoji_mapping.items())[:5])


Sample emoji mapping: []


In [7]:
import nltk

# Redownload 'punkt' tokenizer
nltk.download('punkt', force=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
import nltk
import os

# Manually set the NLTK data directory
nltk_data_path = '/root/nltk_data'
os.makedirs(nltk_data_path, exist_ok=True)
nltk.data.path.append(nltk_data_path)

# Download the 'punkt' resource
nltk.download('punkt', download_dir=nltk_data_path)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
import nltk
import os

# Download 'punkt_tab'
nltk.download('punkt_tab')

# Ensure NLTK data path is set correctly (if needed)
nltk_data_path = '/root/nltk_data'
os.makedirs(nltk_data_path, exist_ok=True)  # Create if doesn't exist
nltk.data.path.append(nltk_data_path)      # Add to NLTK's search path

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Step 5: Tokenize and map text to emoji sequences
def tokenize_text(text):
    tokens = word_tokenize(text.lower())
    return [emoji_mapping.get(token, token) for token in tokens]

# Example text preprocessing
text = "I love sunny weather"
emoji_sequence = tokenize_text(text)
print("Input Text:", text)
print("Emoji Sequence:", emoji_sequence)


Input Text: I love sunny weather
Emoji Sequence: ['i', 'love', 'sunny', 'weather']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load a pre-trained transformer model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Encode input text
input_text = "I love sunny weather"
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

# Generate emoji sequence
outputs = model.generate(inputs["input_ids"], max_length=10, num_return_sequences=1)
emoji_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Emoji Sequence from Model:", emoji_sequence)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Generated Emoji Sequence from Model: i love love love love love love love love


In [16]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.0 MB/s eta 0:00:00


In [17]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Refine emoji selection based on sentiment
def refine_with_sentiment(text, emoji_sequence):
    sentiment = analyzer.polarity_scores(text)
    tone = "positive" if sentiment["compound"] > 0 else "negative"
    refined_sequence = [emoji + ("😊" if tone == "positive" else "😢") for emoji in emoji_sequence]
    return refined_sequence

# Split the emoji sequence into a list of individual emojis
emoji_sequence_list = list(emoji_sequence)

refined_sequence = refine_with_sentiment(input_text, emoji_sequence_list)
print("Refined Emoji Sequence Based on Sentiment:", refined_sequence)

Refined Emoji Sequence Based on Sentiment: ['i😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊', ' 😊', 'l😊', 'o😊', 'v😊', 'e😊']


In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Refine emoji selection based on sentiment
def refine_with_sentiment(text, emoji_sequence):
    sentiment = analyzer.polarity_scores(text)
    tone = "positive" if sentiment["compound"] > 0 else "negative"
    refined_sequence = [emoji + ("😊" if tone == "positive" else "😢") for emoji in emoji_sequence]
    return refined_sequence

refined_sequence = refine_with_sentiment(input_text, emoji_sequence.split())
print("Refined Emoji Sequence Based on Sentiment:", refined_sequence)


Refined Emoji Sequence Based on Sentiment: ['i😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊', 'love😊']


In [20]:
from transformers import AdamW, T5Tokenizer, AutoModelForSeq2SeqLM

# Load a pre-trained T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")  # Use T5 tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Simple training loop (illustration, adjust as needed for your dataset)
for epoch in range(3):  # Iterate through epochs
    model.train()
    inputs = tokenizer("I love sunny weather", return_tensors="pt", max_length=512, truncation=True)
    labels = tokenizer("😊☀️", return_tensors="pt", max_length=10, truncation=True)["input_ids"]

    # Remove token_type_ids from inputs if present
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Epoch {epoch + 1} Loss: {loss.item()}")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_va

Epoch 1 Loss: 4.798637390136719
Epoch 2 Loss: 2.400466203689575
Epoch 3 Loss: 3.6423492431640625


In [21]:
from transformers import AdamW

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Simple training loop (illustration, adjust as needed for your dataset)
for epoch in range(3):  # Iterate through epochs
    model.train()
    inputs = tokenizer("I love sunny weather", return_tensors="pt", max_length=512, truncation=True)
    labels = tokenizer("😊☀️", return_tensors="pt", max_length=10, truncation=True)["input_ids"]

    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Epoch {epoch + 1} Loss: {loss.item()}")


Epoch 1 Loss: 2.541346311569214
Epoch 2 Loss: 2.7545855045318604
Epoch 3 Loss: 1.9193695783615112


In [23]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.0 MB/s eta 0:00:00


In [24]:
from Levenshtein import distance as levenshtein_distance

# Evaluate emoji sequence matching
def evaluate_sequences(predicted, actual):
    score = levenshtein_distance(predicted, actual)
    return score

predicted_sequence = "😊☀️"
actual_sequence = "😊☀️"
score = evaluate_sequences(predicted_sequence, actual_sequence)
print("Levenshtein Distance Between Sequences:", score)

Levenshtein Distance Between Sequences: 0
